In [89]:
# Import the torch library, which is used for various deep learning operations.
import torch

# Import necessary components from the transformers library for working with pre-trained models.
from transformers import AutoModelWithLMHead, AutoTokenizer,AutoModelForSeq2SeqLM

# Initialize a tokenizer for the "google/flan-t5-large" model.
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Initialize a pre-trained "google/flan-t5-large" model for sequence-to-sequence tasks.
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [92]:
# Import the pandas library for data manipulation.
import pandas as pd

# Try reading the CSV file 'DisasterHarveyTweets.csv' with 'utf-8' encoding.
try:
    df = pd.read_csv('DisasterHarveyTweets.csv', encoding='utf-8')
except UnicodeDecodeError:
    # If 'utf-8' encoding doesn't work due to special characters, try using 'iso-8859-1' encoding.
    df = pd.read_csv('DisasterHarveyTweets.csv', encoding='iso-8859-1')


In [93]:
# Select the 'Tweet' column from the DataFrame 'df' and update the DataFrame itself.
# This code reassigns 'df' to contain only the 'Tweet' column, effectively discarding all other columns.
df = df['Tweet']

In [94]:
df

0       Summary for Hurricane Harvey (AT4/AL092017) ht...
1       NASA: RT ESA_EO: Just in! #Sentinel3 measures ...
2       ?Life-threatening? Hurricane Harvey heads towa...
3       Everyone, please pray for Texas. Hurricane Har...
4       Hi Maria, Will Bush Airport & Hobby Expect a S...
                              ...                        
4470    #HurricaneHarvey BE RESPECTFUL when riding thr...
4471    #Shelters #Houston #Hurricane #Harvey http://f...
4472    #MAGA's in Texas!!! The hurricane warnings on ...
4473    Hurricane Harvey Weather Precautions  http://c...
4474    Undocumented migrants who try to flee Hurrican...
Name: Tweet, Length: 4475, dtype: object

In [101]:
# taking only the starting 2000 tweets in its raw form
df1 = df.iloc[:2000]

In [102]:
df1

0       Summary for Hurricane Harvey (AT4/AL092017) ht...
1       NASA: RT ESA_EO: Just in! #Sentinel3 measures ...
2       ?Life-threatening? Hurricane Harvey heads towa...
3       Everyone, please pray for Texas. Hurricane Har...
4       Hi Maria, Will Bush Airport & Hobby Expect a S...
                              ...                        
1995    Mommy and bud are escaping to Tyler Texas.  We...
1996    #BrockLong starring in "Everything's Wetter In...
1997    You know how Hurricane Harvey is different fro...
1998    Oil Prices Rise As Texas Braces For Hurricane ...
1999    Hurricane Harvey is the biggest flood threat o...
Name: Tweet, Length: 2000, dtype: object

In [ ]:
# converting series to a dataframe
df1 = df1.to_frame(name='Tweet')

In [ ]:
# List of emotions
emotions = ['fear', 'anxiety', 'sadness', 'anger', 'helplessness', 'relief', 'frustration', 'confusion', 'hope', 'empathy']

# Loop through each emotion
for emotion in emotions:
    # Create an empty list to store generated answers for the current emotion
    answers = []

    # Define the input question with the current emotion
    input_question = f"Is there {emotion} in the following tweet"

    # Loop through each row in the DataFrame
    for index, row in df1.iterrows():
        text = row['Tweet'] 

        # Concatenate the input question and text
        combined_text = f"{input_question}\n{text}"

        # Generate an answer using the language model
        input_ids = tokenizer(combined_text, return_tensors='pt').input_ids
        output = model.generate(input_ids, max_length=512, num_return_sequences=1)
        ans = tokenizer.decode(output[0], skip_special_tokens=True)

        # Append the generated answer to the list
        answers.append(ans)

    # Add the answers as a new column with the emotion name
    df1[emotion] = answers

In [ ]:
df1

,Tweet,fear,anxiety,sadness,anger,helplessness,relief,frustration,confusion,hope,empathy
0,Summary for Hurricane Harvey (AT4/AL092017) ht...,yes,yes,yes,no,no,no,no,Is this the first tweet from the official Twit...,Is there hope in the tweet,yes
1,NASA: RT ESA_EO: Just in! #Sentinel3 measures ...,no,no,no,no,no,no,no,no,no,no
2,?Life-threatening? Hurricane Harvey heads towa...,yes,yes,yes,no,no,no,no,no,Is there hope in the tweet?Life-threatening? H...,no
3,"Everyone, please pray for Texas. Hurricane Har...",yes,yes,yes,no,no,no,no,no,no,yes
4,"Hi Maria, Will Bush Airport & Hobby Expect a S...",yes,yes,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...
1995,Mommy and bud are escaping to Tyler Texas. We...,yes,yes,yes,no,no,yes,no,no,no,yes
1996,"#BrockLong starring in ""Everything's Wetter In...",yes,yes,no,no,no,no,no,no,I hope so,no
1997,You know how Hurricane Harvey is different fro...,yes,yes,no,no,no,no,no,no,no,yes
1998,Oil Prices Rise As Texas Braces For Hurricane ...,yes,yes,yes,no,no,no,no,no,Is there hope in the tweet Oil Prices Rise As ...,no


In [ ]:
df1.to_csv('harvey_2000_tweets_emotions.csv', index=False)